# 06 – Model Deployment
Deploying the house price prediction model using Streamlit.

In [ ]:
import streamlit as st
import pandas as pd
import joblib
import numpy as np

# Load model and scaler
model = joblib.load("../models/best_house_price_model.pkl")
scaler = joblib.load("../models/scaler.pkl")

# Load data to get feature names
df = pd.read_csv("../data/processed/cleaned_data.csv")
feature_cols = [col for col in df.columns if col != 'price_lkr']

st.title("Sri Lanka House Price Prediction")

# Input fields
district = st.selectbox("District", ["Polonnaruwa", "Matale", "Mullaitivu", "Anuradhapura", "Batticaloa", "Kalutara", "Ratnapura", "Jaffna"])  # Add more as needed
area = st.text_input("Area")
perch = st.number_input("Perch", min_value=0.0)
bedrooms = st.number_input("Bedrooms", min_value=1)
bathrooms = st.number_input("Bathrooms", min_value=1)
kitchen_area = st.number_input("Kitchen Area (sqft)", min_value=0.0)
parking = st.number_input("Parking Spots", min_value=0)
has_garden = st.checkbox("Has Garden")
has_ac = st.checkbox("Has AC")
water_supply = st.selectbox("Water Supply", ["Pipe-borne", "Well", "Both"])
electricity = st.selectbox("Electricity", ["Single phase", "Three phase"])
floors = st.number_input("Floors", min_value=1)
house_age = st.number_input("House Age", min_value=0)

if st.button("Predict Price"):
    # Prepare input
    input_data = {
        'perch': perch,
        'bedrooms': bedrooms,
        'bathrooms': bathrooms,
        'kitchen_area_sqft': kitchen_area,
        'parking_spots': parking,
        'has_garden': int(has_garden),
        'has_ac': int(has_ac),
        'floors': floors,
        'house_age': house_age,
        'total_rooms': bedrooms + bathrooms,
        'area_per_room': perch / (bedrooms + bathrooms) if bedrooms + bathrooms > 0 else 0,
        'luxury_score': int(has_garden) + int(has_ac) + int(parking > 1)
    }
    
    # One-hot encode categoricals
    for col in feature_cols:
        if col.startswith('district_'):
            input_data[col] = 1 if f"district_{district}" == col else 0
        elif col.startswith('area_'):
            input_data[col] = 1 if f"area_{area}" == col else 0
        elif col.startswith('water_supply_'):
            input_data[col] = 1 if f"water_supply_{water_supply}" == col else 0
        elif col.startswith('electricity_'):
            input_data[col] = 1 if f"electricity_{electricity}" == col else 0
        else:
            input_data.setdefault(col, 0)
    
    input_df = pd.DataFrame([input_data])
    input_df = input_df[feature_cols]  # Ensure order
    
    # Scale if needed
    if hasattr(model, 'n_estimators'):  # Tree-based, no scaling
        prediction = model.predict(input_df)
    else:
        input_scaled = scaler.transform(input_df)
        prediction = model.predict(input_scaled)
    
    st.success(f"Predicted Price: LKR {prediction[0]:,.0f}")